# Data Journalism Lesson 17: Line charts

Showing how things change over time with lines.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('plotly')
await micropip.install("nbformat>=4.2.0")

In [ ]:
from IPython.display import display, HTML
import pandas as pd

# --- Simple Grading/Checking Functions ---
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_df_exists(df, df_name):
    if isinstance(df, pd.DataFrame) and not df.empty:
        display_feedback(True, f'{df_name} DataFrame loaded successfully and is not empty.', '')
    else:
        display_feedback(False, f'{df_name} DataFrame is not loaded correctly or is empty.', 'Please check the loading process.')

def check_column_exists(df, column_name, df_name):
    if column_name in df.columns:
        display_feedback(True, f'Column \'{column_name}\' exists in {df_name}.', '')
    else:
        display_feedback(False, f'Column \'{column_name}\' does not exist in {df_name}.', 'Please check your column creation steps.')

def check_date_column(df, column_name, df_name):
    if column_name in df.columns and pd.api.types.is_datetime64_any_dtype(df[column_name]):
        display_feedback(True, f'Column \'{column_name}\' in {df_name} is correctly formatted as a datetime.', '')
    elif column_name not in df.columns:
        display_feedback(False, f'Column \'{column_name}\' does not exist in {df_name}.', 'Please ensure the column is created.')
    else:
        display_feedback(False, f'Column \'{column_name}\' in {df_name} is NOT formatted as a datetime.', f'Its current type is {df[column_name].dtype}. Use pd.to_datetime().')

def check_plot_params(params, expected_params, plot_name):
    correct = True
    messages = []
    for p_name, p_val in expected_params.items():
        if params.get(p_name) == p_val:
            messages.append(f'Correct {p_name} for {plot_name}.')
        elif isinstance(params.get(p_name), pd.DataFrame) and isinstance(p_val, pd.DataFrame):
             if params.get(p_name).equals(p_val):
                 messages.append(f'Correct DataFrame for {p_name} in {plot_name}.')
             else:
                 correct = False
                 messages.append(f'Incorrect DataFrame for {p_name} in {plot_name}. Content differs.')
        else:
            correct = False
            messages.append(f'Incorrect {p_name} for {plot_name}. Expected \'{p_val}\', got \'{params.get(p_name)}\'.')
    final_message_correct = f'Plot parameters for {plot_name} are correct!'
    final_message_incorrect = ' '.join(messages)
    display_feedback(correct, final_message_correct, final_message_incorrect)

In [ ]:
# --- State Setup and Data Loading ---
state_abbr = 'MN' # Default from RMD, can be changed
state_full_name = "Minnesota"

# Construct data URL (simplified logic based on R's case_when which essentially formats the state name)
data_url = f"../_static/unemployment-rates/{state_full_name.lower().replace(' ', '-')}.csv"
national_data_url = "../_static/unemployment-rates/national.csv"

try:
    unemployment_df_initial = pd.read_csv(data_url)
    national_df_initial = pd.read_csv(national_data_url)
except Exception as e:
    print(f"Error loading unemployment data: {e}. Using empty DataFrames as fallback.")
    unemployment_df_initial = pd.DataFrame()
    national_df_initial = pd.DataFrame()

unemployment_rows_expected = len(unemployment_df_initial)


# Pre-process state unemployment data for exercises (real_date column)
if not unemployment_df_initial.empty:
    state_unemployment_df_processed = unemployment_df_initial.copy()
    state_unemployment_df_processed['combine_date'] = state_unemployment_df_processed['periodName'] + ' 1, ' + state_unemployment_df_processed['year'].astype(str)
    state_unemployment_df_processed['real_date'] = pd.to_datetime(state_unemployment_df_processed['combine_date'], format='%B %d, %Y')
else:
    state_unemployment_df_processed = pd.DataFrame(columns=['Location', 'year', 'period', 'periodName', 'Unemployment_Rate', 'combine_date', 'real_date'])

# Pre-process national data similarly if it has the same structure
if not national_df_initial.empty:
    national_df_processed = national_df_initial.copy()
    if 'periodName' in national_df_processed.columns and 'year' in national_df_processed.columns:
        national_df_processed['combine_date'] = national_df_processed['periodName'] + ' 1, ' + national_df_processed['year'].astype(str)
        national_df_processed['real_date'] = pd.to_datetime(national_df_processed['combine_date'], format='%B %d, %Y')
    # If national data already has a 'real_date' or different structure, adjust accordingly
    # For this RMD, it's implied to be the same structure and needs date processing.
else:
    national_df_processed = pd.DataFrame(columns=['Location', 'year', 'period', 'periodName', 'Unemployment_Rate', 'combine_date', 'real_date'])


In [ ]:
from myst_nb import glue

glue("state_full_name", state_full_name, display=False)

## The Goal

In this lesson, you'll learn how to create line charts to visualize change over time. By the end of this tutorial, you'll understand when to use line charts, how to prepare your data for this type of visualization, and how to create both single-line and multi-line charts using Plotly Express. You'll practice filtering data, grouping by time periods, and adding aesthetic elements like color to differentiate multiple lines. These skills will enable you to effectively communicate trends and patterns in time-series data, a crucial ability for data journalists.

## Why Visualize Data?

While often incorrectly credited with creating graphical forms, there are few on earth who can claim more credit for popularizing certain forms than William Playfair. 

Born in 1759 as the fourth son of a Scottish clergyman, Playfair was a millwright apprentice for a while, a draughtsman later, an inventor of no note and a not-very-successful businessman. But starting in 1786, aged 27, he started to make a living as a writer. His stock-in-trade were such thrilling books as The Commercial and Political Atlas (all three editions!), Lineal Arithmetic and An Inquiry into the Permanent Causes of the Decline and Fall of Powerful and Wealthy Nations. 

So why are we talking about these dry texts centuries later? Because they had charts in them. 

```{image} ../_static/images/playfair.png
:alt: Playfair Chart Example
:width: 600px
:align: center
```

Charts were not common to see in print, and they weren't widely used as a means of improving understanding for a general reader. They did exist. But Playfair popularized their use and wrote often about using them to help people understand complex topics. In 1786, he wrote that busy people need visual aids so they can absorb information "without the fatigue and trouble of studying the particulars of which it is composed." 

Imagine how Playfair would feel about modernity if people in 1786 are overwhelmed with information.

The graphical form that Playfair enters the most discussions about is the line chart, which we learn about in this chapter. The line chart shows change over time, and gives a reader a chance to absorb those changes between entities and time. Playfair wrote in 1805 that such charts even helped him understand what he was writing about better than all of his studies. 

"I first drew the Chart in order to clear up my own ideas on the subject, finding it very troublesome to retain a distinct notion of the changes that had taken place," Playfair wrote in a footnote in An Inquiry into the Permanent Causes of the Decline and Fall of Powerful and Wealthy Nations. "I found it answer(ed) the purpose beyond my expectation, by bringing into one view the result of details that are dispersed over a very wide and intricate field of universal history; facts sometimes connected with each other, sometimes not, and always requiring reflection each time they were referred to. 

"I found the first rough draft (gave) me a better comprehension of the subject, than all that I had learnt from occasional reading, for half of my lifetime; and, on the supposition that what was of so much use to me, might be of some to others, I have given it with a tolerable degree of accuracy."

May we all make charts that improve understanding that much.

## The Basics

So far, we've talked about bar charts -- normal or waffle -- which are good for showing relative size of a thing compared to another thing. Waffle charts are good at showing proportions of a whole. 

**Line charts are good for showing change over time.** 

Let's look at some recent history and some unemployment data and look at just how bonkers March of 2020 was. It seems like the world just stopped that month because of a global pandemic. You can see it in all kinds of data -- and we'll be studying that time period for years to come. 

We'll start with libraries. Good news: We don't need much for this one beyond `pandas` for data manipulation and `plotly.express` for plotting.

In [ ]:
import pandas as pd
import plotly.express as px

Now import the data. This is data from the Bureau of Labor Statistics of state unemployment rates from 2019 through the middle of 2024.

In [ ]:
unemployment_df = pd.read_csv("../_static/unemployment-rates/Minnesota.csv")
display(unemployment_df.head())

Line charts show change over time. It works much the same as a bar chart, code wise, but instead of categories on the x-axis, you typically have dates or time periods. If you have more than one group in your data (e.g., multiple states or categories), Plotly Express can create different lines for each group, often by using the `color` argument.

The first secret to knowing if you have a line chart is if you have data that changes over time. The second secret to making a line chart is your x-value is almost always a date. If it's not an actual date, it's a thing like a date -- a year, a season, a semester, something with date elements in it. 

Let's look at how unemployment changed over the the past five years -- from before the pandemic until more recent months. First, take a look at our data. You should see a problem right away.

In [ ]:
display(____.head())

See it? We have *monthly* data, but the data are in parts. March of 2020 is one line, April of 2020 is another line, but the month and the year are in two separate columns (`periodName` and `year`). And *neither* are a date type that Plotly can easily use for a time-series x-axis.

This is precisely why there is a working with dates chapter (or section in many data analysis courses!). Every data system on the planet has a slightly different idea of how to store dates. Every system has their own needs, and those needs are *rarely* ever aligned with yours.

How do we fix this?

We're going to do this in two steps. The first step is creating a string that looks like a full date -- we're going to make every date the first of the month. A date in pandas (and for Plotly) needs to be recognizable as month, day, and year. The first of the month is as good as any other day for monthly data. We'll do that by combining the `periodName` column, the text string " 1, " (note the spaces and comma), and the `year` column.

### Exercise 1: Making a not-real date string

Create a new column called `combine_date` in `unemployment_df` by concatenating `periodName`, the string " 1, ", and `year`. Remember to convert `year` to string type if it's not already.

In [ ]:
unemployment_df['combine_date'] = unemployment_df[____] + ____ + unemployment_df[____].astype(str)

display(unemployment_df[['periodName', 'year', 'combine_date']].head())
check_column_exists(unemployment_df, 'combine_date', 'unemployment_df')

From this, you can see we now have what to you and me *looks* like a date string, but to pandas and Plotly, it's just text. For example, "January 1, 2020" is not yet a proper datetime object.

Recall from working with dates that if the date string follows a consistent pattern, we can convert it using `pd.to_datetime()` and specifying the `format` argument. The format codes tell pandas how to interpret the string parts.

Common format codes include:

| Code | Meaning                                        |
|------|------------------------------------------------|
| `%a` | Abbreviated weekday (Mon, Tue, Wed, etc)       |
| `%A` | Full weekday (Monday, Tuesday, Wednesday, etc) |
| `%b` | Abbreviated month (Jan, Feb, Mar, etc.)        |
| `%B` | Full month (January, February, etc.)           |
| `%c` | Locale-specific date and time                  |
| `%d` | Day of the month as a zero-padded decimal.     |
| `%H` | Decimal hours (24 hour)                        |
| `%I` | Decimal hours (12 hour)                        |
| `%j` | Decimal day of the year                        |
| `%m` | Month as a zero-padded decimal number.         |
| `%M` | Decimal minute                                 |
| `%p` | Locale-specific AM/PM                          |
| `%S` | Decimal second                                 |
| `%U` | Decimal week of the year (starting on Sunday)  |
| `%w` | Decimal Weekday (0=Sunday)                     |
| `%W` | Decimal week of the year (starting on Monday)  |
| `%x` | Locale-specific Date                           |
| `%X` | Locale-specific Time                           |
| `%Y` | Year with century as a decimal number.         |
| `%y` | Year without century as a zero-padded decimal number.|
| `%z` | Offset from GMT                                |
| `%Z` | Time zone (character)                          |

Our string is like "Month Day, Year" (e.g., "January 1, 2020"). We need full month name, day of the month, and 4-digit year.

### Exercise 2: Making a real date

Convert the `combine_date` column to a datetime object using `pd.to_datetime()` and store it in a new column called `real_date`. Specify the correct `format` string.

In [ ]:
unemployment_df['real_date'] = pd.to_datetime(unemployment_df[____], format="____")

display(unemployment_df[['combine_date', 'real_date']].head())

check_date_column(unemployment_df, 'real_date', 'unemployment_df')

And, as you can see, you now have a real date column called `real_date` with a datetime64[ns] dtype. Now we have the two things we need to make a line chart -- a date (`real_date`) and a number (`Unemployment_Rate`).

### Exercise 3: Making a line chart

The DataFrame `state_unemployment_df_processed` already has the `real_date` column processed for you. Use this DataFrame to create a line chart with Plotly Express.

The x-axis should be `real_date` and the y-axis should be `Unemployment_Rate`.

In [ ]:
fig_ex3 = px.line(
    data_frame=____, 
    x=____, 
    y=____,
    title=f"Unemployment Rate in {state_full_name}"
)
fig_ex3.show()

So what story does this tell? Looks like before the pandemic, unemployment was humming along reasonably stable, and then boom. The pandemic hits and the unemployment rate balloons. Then, after a bit, it returns to more normal ranges.

But a one-line line chart is usually not a line chart telling a complete story. At least, not as complete as it could be. We need more to compare to.

The DataFrame `national_df_processed` (also created behind the scenes) contains unemployment data for all states, with the `real_date` column already processed. It's structured similarly to `state_unemployment_df_processed` but includes data for many locations. Take a look for yourself.

In [ ]:
display(____.head())

### Exercise 4: More than one line.

A critical concept to understand in visualizing data is the idea of **layering** or grouping. With Plotly Express, we can create multiple lines on the same chart, each representing a different category (like a state). This is often done by mapping a column to the `color` argument. Plotly Express will then draw a separate line for each unique value in that column.

Let's create a line chart showing unemployment rates for all locations in the `national_df_processed` DataFrame. The `Location` column indicates the state or area. Use this for the `color`.

In [ ]:
fig_ex4 = px.line(
    data_frame=____, 
    x='real_date', 
    y='Unemployment_Rate', 
    color=____,
    title="Unemployment Rates by State"
)
fig_ex4.show()

What do you see here? In the trade, we call this the hairball. Lines overlapping, a big mess. You can see the pandemic hitting, clear as a bell, but what else are you supposed to look at? What happened to your state? How does it compare to the rest of fooling around?

And now you know why effective layering or highlighting is important.

First, let's do some things to fade the hairball into the background a little. With Plotly, after creating the figure, we can iterate through its traces (lines) and update their properties. For lines we want to fade, we can change their color to grey and reduce their opacity or width.

However, a more direct Plotly Express approach for de-emphasizing lines is often to prepare the data slightly differently or apply styles post-creation carefully. For this exercise, we'll use `color_discrete_sequence` to set all lines to grey initially, and then we'll think about how to highlight a specific state.

### Exercise 5: Fading it back

Re-create the multi-line chart from Exercise 4, but this time, make all lines a light grey color. You can use the `color_discrete_sequence` argument in `px.line()`. You'll also need to specify `line_group='Location'` if `color` is not mapped to `Location`, to ensure separate lines are drawn per location when using a single color sequence for all.

In [ ]:
fig_ex5 = px.line(
    data_frame=____, 
    x=____, 
    y=____, 
    line_group=____, # Ensure lines are grouped by Location
    color_discrete_sequence=____, # e.g., ['lightgrey']
    title="National Unemployment Rates (Faded)"
)
fig_ex5.show()

Better for a background. Now we need to add our specific state's line back, but this time make it stand out. In Plotly, we can add traces to an existing figure. We'll create the base figure with all national lines faded (as in Exercise 5), and then add a new trace for our specific state, {glue:text}`state_full_name`, in a prominent color like red.

### Exercise 6: Highlighting Your State

1. Create the faded national unemployment chart as in Exercise 5. Store it in `fig_ex6`.
2. Use `state_unemployment_df_processed` (which contains data for {glue:text}`state_full_name`) to create a line trace.
3. Add this new trace to `fig_ex6` using `fig_ex6.add_traces()`. You'll need to get the data component of a `px.line()` call for your state.
4. Manually set the color of this new trace to 'red' and give it a name for the legend.

In [ ]:
# Step 1: Create the faded national chart (can reuse fig_ex5 logic or redefine)
fig_ex6 = px.line(
    national_df_processed, 
    x='real_date', 
    y='Unemployment_Rate', 
    line_group='Location',
    color_discrete_sequence=['lightgrey'],
    title=f"Unemployment Rate: {state_full_name} vs. Nation"
)
fig_ex6.update_traces(line=dict(width=0.5)) # Make background lines thin

# Step 2 & 3: Create and add the highlighted state trace
highlight_trace = px.line(
    data_frame=____, # DataFrame for your specific state (state_unemployment_df_processed)
    x=____, 
    y=____
).data[0] # Get the trace data object

# Step 4: Customize and add the trace
highlight_trace.line.color = ____ # e.g., 'red'
highlight_trace.name = state_full_name # Set name for legend
highlight_trace.line.width = 2 # Make highlighted line thicker

fig_ex6.add_traces(highlight_trace)
fig_ex6.update_layout(showlegend=True) # Ensure legend is shown, may need further legend customization
fig_ex6.show()

And now we ask some critical questions: Does this tell a story? What is that story? How does your state compare to the nation, during the pandemic and after?

## The Recap

Throughout this lesson, you've learned the fundamentals of creating line charts to show change over time using Plotly Express. You've practiced preparing date data, creating basic single-line charts, and expanding to multi-line charts to compare different states over time. You've seen how line charts can reveal trends, such as the impact of the pandemic on employment across the country, and how to highlight specific series for comparison. Remember, while line charts are powerful tools for visualizing temporal data, they're just the starting point. The patterns you discover should lead to further questions and deeper reporting to uncover the full story behind the numbers.

## Terms to Know

- **Line chart**: A type of graph that displays data points connected by straight line segments, typically used to show trends over time.
- **`plotly.express.line()` (or `px.line()`)**: A Plotly Express function used to create line charts.
- **`x`, `y` arguments**: In `px.line()`, these map DataFrame columns to the x (time) and y (value) axes.
- **`color` argument**: In `px.line()`, used to assign different colors to lines based on categories in a specified column (e.g., different states).
- **`line_group` argument**: In `px.line()`, explicitly defines how data should be grouped to form lines, especially useful if `color` is not used for grouping or if multiple lines share a color.
- **`color_discrete_sequence`**: An argument in Plotly Express functions to specify a list of colors to use for categorical data.
- **`pd.to_datetime()`**: A pandas function to convert argument to datetime.
- **Format String (for dates)**: A string of codes (e.g., `"%B %d, %Y"`) that specifies how to interpret a date string.
- **Time-series data**: Data that is collected at different points in time, allowing for analysis of trends and patterns over time.
- **`fig.add_traces()`**: A Plotly figure method to add one or more traces (like lines from another plot) to an existing figure.
- **`fig.update_traces()`**: A Plotly figure method to modify properties of existing traces in a figure.